In [5]:
#import package
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pylab as plt
import time

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM, GRU

from keras import optimizers
#%matplotlib inline

print('Done')

Done


In [2]:
def readData():
    data = pd.read_csv('C:/Users/yoon/Python/Stock_Prediction/Samsung_20_DIV.csv', parse_dates = [0])
    raw_dates = data.loc[30:, 'date'].reset_index(drop=True)
    del data['date']
    raw_data = data.loc[30:].reset_index(drop=True)
    return raw_data, raw_dates

def split_into_chunks(data, train, predict, step):
    train_set, label_set = [], []
    for i in range(0, len(data), step):
        try:
            temp_train = data.loc[i:i+train-1, 'dSMA':'dCCI']
            temp_train = preprocessing.scale(temp_train)
            temp_label = data.loc[i+train-1+predict, 'return']
            if temp_label >= 0 :
                 temp_label = [1., 0.]
            else:
                 temp_label = [0., 1.]
        except:
            break
        train_set.append(temp_train)
        label_set.append(temp_label)
    return train_set, label_set

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)
    return a, b

def create_Xt_Yt(X, y, percentage=0.5):
    X_train = X[0:int(len(X) * percentage)]
    Y_train = y[0:int(len(y) * percentage)]
    X_train, Y_train = shuffle_in_unison(X_train, Y_train)
    X_test = X[int(len(X) * percentage):]
    Y_test = y[int(len(X) * percentage):]
    return X_train, X_test, Y_train, Y_test

print('Done')

Done


In [3]:
timeSeriesList, dates = readData()

DATA_SIZE_ON_TRAIN = 30
TARGET_TIME = 1
STEP_SIZE = 1
EMB_SIZE = 10

X, Y = split_into_chunks(timeSeriesList, DATA_SIZE_ON_TRAIN, TARGET_TIME, STEP_SIZE)
X, Y = np.array(X), np.array(Y)
X_train, X_test, Y_train, Y_test = create_Xt_Yt(X, Y, percentage=0.5)

print('Done')

Done


In [ ]:
#reshape (#samples, #time steps(time series), #feature)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], EMB_SIZE))
#X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], EMB_SIZE))
model = Sequential()

#input_shape = (#time step, #feature)
model.add(LSTM(50, input_shape = (DATA_SIZE_ON_TRAIN, EMB_SIZE), dropout = 0.5, return_sequences=True))
model.add(LSTM(2, return_sequences=False))

adam = optimizers.Adam(lr=0.1)

model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("Training...")
model.fit(X_train, 
          Y_train, 
          epochs=100, 
          batch_size = 128,
          verbose=1)

print("Evaluating...")
score = model.evaluate(X_test, Y_test, batch_size=128)
print("\nTest loss and accuracy:" + str(score))

print('Done')

Training...
Epoch 1/100
2579/2579 [==============================] - 20s - loss: 4.6594 - acc: 0.4827    
Epoch 2/100
1792/2579 [===================>..........] - ETA: 2s - loss: 4.7236 - acc: 0.4911